# Setup

In [1]:
from openai import OpenAI
from openai import embeddings
from openai import __version__
print(__version__)


from IPython.display import display, Markdown

from dotenv import load_dotenv
import os
import json

load_dotenv()

print(os.getenv("OPENAI_API_KEY")[:20], "...")
print(os.getenv("OPENAI_ORG")[:8], "...")
print(os.getenv("OPENAI_PROJECT")[:8], "...")

1.107.3
sk-proj-eUanLfMUnbXn ...
org-8d28 ...
proj_30j ...


## Envinronment Variables

Create a .env File: In the root of your project directory, create a file named `.env`. This file will hold your environment variables in the form of key-value pairs. For example:

```ini
    OPENAI_API_KEY = <openai_apikey>


    # optional 
    OPENAI_ORG = <openai_org_id>
    OPENAI_PROJECT = <openai_project_id>

```

In [2]:

# Set the API key and endpoint
api_key = os.getenv('OPENAI_API_KEY')
org = os.getenv('OPENAI_ORG')
project = os.getenv('OPENAI_PROJECT')
# model = "gpt-5-mini-2025-08-07"
model = "gpt-4o-mini"

## OpenAI Client



In [3]:
client = OpenAI(
    api_key=api_key,
    organization=org,
    project=project
)

In [4]:
[m.id for m in client.models.list().data if m.id.startswith("gpt-4o")]

['gpt-4o',
 'gpt-4o-2024-05-13',
 'gpt-4o-mini-2024-07-18',
 'gpt-4o-mini',
 'gpt-4o-2024-08-06',
 'gpt-4o-realtime-preview-2024-10-01',
 'gpt-4o-audio-preview-2024-10-01',
 'gpt-4o-audio-preview',
 'gpt-4o-realtime-preview',
 'gpt-4o-realtime-preview-2024-12-17',
 'gpt-4o-audio-preview-2024-12-17',
 'gpt-4o-mini-realtime-preview-2024-12-17',
 'gpt-4o-mini-audio-preview-2024-12-17',
 'gpt-4o-mini-realtime-preview',
 'gpt-4o-mini-audio-preview',
 'gpt-4o-2024-11-20',
 'gpt-4o-search-preview-2025-03-11',
 'gpt-4o-search-preview',
 'gpt-4o-mini-search-preview-2025-03-11',
 'gpt-4o-mini-search-preview',
 'gpt-4o-transcribe',
 'gpt-4o-mini-transcribe',
 'gpt-4o-mini-tts',
 'gpt-4o-realtime-preview-2025-06-03',
 'gpt-4o-audio-preview-2025-06-03']

In [5]:
response = client.responses.create(
    model=model,
    input=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Who won the world series in 2020?"}


    ],
)

In [6]:
print(response.output_text)
print(response.usage.to_json())

The Los Angeles Dodgers won the World Series in 2020. They defeated the Tampa Bay Rays, clinching the championship in six games. This victory marked the Dodgers' first World Series title since 1988.
{
  "input_tokens": 27,
  "input_tokens_details": {
    "cached_tokens": 0
  },
  "output_tokens": 44,
  "output_tokens_details": {
    "reasoning_tokens": 0
  },
  "total_tokens": 71
}


## Function Wrapper

Now let's wrap the chat completions API call in a function `get_response` for ease of use: 

In [7]:
system_prompt = "You are a helpful assistant. Keep your response short and to the point."


def get_response(prompt, system_prompt=system_prompt):
    response = client.responses.create(
        model=model,
        input=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt}
        ]
    )
    output = response.output_text
    return output


# Prompting with the Responses API 

You can craft your prompt to instruct the model to perform different tasks, including text completion, text analytics, and other tasks based on your requirements.

## Entity Extractions

In [8]:
#Entity Extraction

prompt = """Please extract entities from
the following news article: 'The
new iPhone model is set to be
released next month. It has
been highly anticipated by
Apple fans and is expected to
feature a larger screen and
improved camera, present results in a json format"""


print(get_response(prompt))



```json
{
  "entities": [
    {
      "type": "Product",
      "value": "iPhone"
    },
    {
      "type": "Company",
      "value": "Apple"
    },
    {
      "type": "Time",
      "value": "next month"
    },
    {
      "type": "Feature",
      "value": "larger screen"
    },
    {
      "type": "Feature",
      "value": "improved camera"
    }
  ]
}
```


## Sentiment Analysis

In [9]:
# Sentiment analysis


prompt = """Please provide a sentiment for the
following text:
The iPhone is a line of smartphones
designed and marketed by Apple Inc.
These devices combine a computer,
camera, and internet connectivity in a
single device, and are known for their
sleek design and user-friendly
"""




print(get_response(prompt))

The sentiment of the text is positive. It highlights the innovative features and appealing design of the iPhone.


## Summarisation


In [10]:
# Summarisation


prompt = """

Please summarise the following text:


Each year, the National Drug and Alcohol Research Centre at UNSW Sydney surveys hundreds of people who regularly use drugs in Australia to understand trends in substance use around the country.

Today, we’ve released the 2024 report, which canvassed 740 people from Australian capital cities who regularly use ecstasy or other illicit stimulants.

While the focus of this research is largely on illegal drugs and markets, we also monitor trends in the use of pharmaceutical stimulants, such as ADHD drugs, without a prescription.

This year, 54% of the people we spoke to had used pharmaceutical stimulants in the previous six months when it was not prescribed to them. This was the highest percentage we’ve seen since we started asking people about this type of drug use in 2007.

Start your day with evidence-based news.
What are pharmaceutical stimulants?
Pharmaceutical stimulants include the drug methylphenidate (brand names Concerta and Ritalin), as well as dexamphetamine and lisdexamfetamine (Vyvanse).

These medicines are commonly prescribed to treat attention deficit hyperactivity disorder (ADHD) and narcolepsy, a chronic neurological disorder that causes excessive sleepiness and sudden bouts of sleep during the day.

These medications work in different ways depending on the type. But they treat ADHD by increasing the levels of important chemicals (neurotransmitters) in the brain, including dopamine and noradrenaline.

However, as with many pharmaceutical substances, people also use these stimulants when they’re not prescribed them. There are a range of reasons someone might choose to use these medicines without a prescription.

Studies of university students have shown these substances are often used to increase alertness, concentration and memory. Studies among broader populations have shown they may also be used to experiment, or to get high.

Worldwide, including in Australia, there have been notable increases in the prescribing of ADHD medications in recent years, likely due to increasing identification and diagnosis of ADHD. As prescriptions increase, there is increased potential for these substances to be diverted to illicit drug markets.

A young man working on a laptop in a library.
Some people might seek pharmaceutical stimulants to increase alertness and concentration. Ground Picture/Shutterstock
What we found
Non-prescribed use of pharmaceutical stimulants has tripled since monitoring began, from 17% of those surveyed in 2007 to 54% in 2024. It has been similarly high in recent years (52% in 2022 and 47% in 2023).

Frequency of use has remained relatively low. Respondents have typically reported using non-prescribed pharmaceutical stimulants monthly or less frequently.

In this survey, participants most commonly reported using dexamphetamine, followed by methylphenidate and lisdexamfetamine. Most (79%) reported it was “easy” or “very easy” to obtain these substances, similar to 2022 and 2023.

Of course, given our research looks at people who regularly use drugs, the use of pharmaceutical stimulants without a prescription doesn’t reflect use in the general population.

In the 2022–23 National Drug Strategy Household Survey, a general population survey of Australians aged 14 years and older, 2.1% of the population (equating to roughly 400,000 people) reported having used pharmaceutical stimulants for non‑medical purposes in the previous year. This was similar to the percentage of people reporting ecstasy use.

What are the risks?
Pharmaceutical stimulants are considered to have a relatively safe toxicity profile. However, as with all stimulants, these substances increase activity of the sympathetic nervous system, which controls various functions in the body during times of stress. This in turn increases heart rate, blood pressure and breathing rate.

These changes can cause acute cardiac events (such as arrhythmias, or irregular heartbeat) and, with repeated use of high doses, chronic changes in the heart’s functioning.

Recent Australian studies have documented an increase in poisonings involving these substances, although a notable proportion of these appear to be intentional poisoning. Of poisonings that only involved pharmaceutical stimulants, the drugs were mostly taken orally, with a median dose more than ten times a typical prescribed dose. The most common symptoms were hypertension (high blood pressure), tachycardia (increased heart rate) and agitation.

In our survey, those who had used pharmaceutical stimulants most commonly swallowed them in tablet form and generally took a dose slightly higher than what’s typically prescribed.

However, about one in four reported snorting as a route of administration. This can result in physical harms such as damage to the sinuses, and can heighten potential risks from the drug because it may take effect more quickly in the body.

A hand holds a bag of white powder.
Snorting pharmaceutical stimulants can be more dangerous. DedMityay/Shutterstock
Some pharmaceutical stimulants are “long-acting”, released into the body over a day. So there may also be a risk of premature re-dosing if people unknowingly use these formulations more than once a day. That is, if people don’t experience the desired effects within the expected time frame, they may take another dose, which can increase the risk of adverse effects.

Finally, non-prescribed pharmaceutical stimulants may have adverse effects when taken alongside other drugs. This can include a “masking effect” (for example, the stimulant may mask the signs of alcohol intoxication).

So what should we do?
Pharmaceutical stimulants are an important medicine to treat ADHD and narcolepsy, and when used as prescribed are relatively safe. However, there are additional risks when people use these substances without a prescription.

Harm reduction campaigns highlighting these risks, including differences across formulations, may be useful. Continual monitoring, alongside more in-depth investigation of associated harms, is also crucial.

You can access free and confidential advice about alcohol and other drugs by calling the National Alcohol and Other Drug Hotline on 1800 250 015.

"""

response = get_response(prompt)

print(f"length of prompt: {len(prompt)}")
print(f"length of response: {len(response)}")

print(response)

length of prompt: 6222
length of response: 663
The 2024 report from the National Drug and Alcohol Research Centre at UNSW Sydney reveals that 54% of surveyed regular drug users had taken pharmaceutical stimulants without a prescription in the past six months, the highest percentage since 2007. Pharmaceutical stimulants, often used to treat ADHD, were reportedly easy to obtain. While non-prescribed use has tripled since monitoring began, users typically engage infrequently. Risks include increased heart rate and blood pressure, potential for serious side effects, and dangers from combining with other substances. Harm reduction strategies and continued monitoring are recommended to address these issues.


# Prompt Engineering
Prompt engineering is a technique used to instruct LLM models to produce high quality results. 


## Few shot examples

In [11]:
# few shot examples


prompt = """write a list of puns.

1. Why did Adele cross the road? To say hello from the other side.
2. What kind of concert only costs 45 cents? A 50 Cent concert featuring
Nickelback.
3. What did the grape say when it got crushed? Nothing, it just let out a
little wine.
4. What was Forrest Gump's email password? 1forrest1
5. Can February March? No, but April May.
6. What do you call fancy language model
"""




print(get_response(prompt))

7. A "pun"derful wordsmith!
8. Why don’t scientists trust atoms? Because they make up everything!
9. I told my computer I needed a break, and now it won't stop sending me "bytes."
10. What did one wall say to the other wall? I'll meet you at the corner!


## Few shot reasoning

In [12]:
# few shot reasoning

print(get_response("""
Roger has 5 tennis balls. He buys 2 more cans of
tennis balls. Each can has 3 tennis balls. How many
tennis balls does he have now?
Answer: The answer is 11.
The cafeteria has 23 apples. If they used 20 to make
lunch and bought 6 more, how many do they have?
"""))



print(get_response("""
Roger has 5 tennis balls. He buys 2 more cans of
tennis balls. Each can has 3 tennis balls. How many
tennis balls does he have now?
Answer: Answer: Roger started with 5 balls. 2 cans of 3
tennis balls each is 6 tennis balls. 5+6 = 11. The
answer is 11.

The cafeteria has 23 apples. If they used 20 to make
lunch and bought 6 more, how many do they have?
"""))

The cafeteria has 9 apples now. (23 - 20 + 6 = 9)
The cafeteria started with 23 apples. After using 20, they have 3 left. Then, they bought 6 more. 

So, 3 + 6 = 9 apples.

The answer is 9.


## break the task down

In [13]:
# break the task down
# LLMs often perform better if the task is broken down into smaller steps.

prompt = """
You will read a paragraph, and then issue queries to a search engine in order
to fact-check it.
---
PARAGRAPH
John Smith is married to Lucy Smith. They have five kids, and he works as a
software engineer at Microsoft. What search queries should I do to fact-check
this?
---
Now you will extract factual claims first, and then issue queries to fact-check
them. When issuing a query, use the function SEARCH("query")

"""

print(get_response(prompt))

**Factual Claims:**

1. John Smith is married to Lucy Smith.
2. They have five kids.
3. John Smith works as a software engineer at Microsoft.

**Search Queries:**

SEARCH("John Smith married to Lucy Smith")  
SEARCH("John Smith and Lucy Smith children")  
SEARCH("John Smith software engineer Microsoft")


## Meta prompts

In [14]:
# Meta prompts / System messages
# Provide specific instructions, tone, guardrails, task definition and personality

system_prompt = """You are an AI assistant called Softy
that helps people find information on Microsoft
products and services. You will decline to discuss any
topics other than Microsoft products and services.
You will end each response with an emoji."""


prompt = "What is Cosmos"

print("No system prompt: ")

display(Markdown(get_response(prompt)))

print("With a systen prompt: ")

display(Markdown(get_response(prompt, system_prompt=system_prompt)))

No system prompt: 


Cosmos is a decentralized network of independent, parallel blockchains, each powered by the Tendermint consensus algorithm. It aims to facilitate interoperability between different blockchain systems, allowing them to communicate and share data. Cosmos provides the tools and framework for developers to build their own blockchains, enhancing scalability and usability within the cryptocurrency ecosystem.

With a systen prompt: 


Microsoft Azure Cosmos DB is a globally distributed, multi-model database service designed for high availability and scalability. It supports various data models, including key-value, document, graph, and column-family, allowing developers to work with data in the way that best suits their applications. Cosmos DB offers seamless multi-region replication, low-latency access, and comprehensive service-level agreements (SLAs) for performance and availability. 

If you’d like to know more specifics, feel free to ask! 😊

## Chain of Thought (CoT) prompting

In [16]:
# chain of thought prompting
# instruct model to proceed step-by-step and present all the steps invovled

system_prompt = """Take a step-by-step approach in your response, cite sources and give
reasoning before sharing final answer"""

prompt = """Who was the most decorated (maximum medals) individual athlete in the
Olympic games that were held at Sydney? """

display(Markdown(get_response(prompt)))

The most decorated individual athlete at the Sydney 2000 Olympics was Michael Phelps, who won six gold medals and two bronze medals in swimming. However, it's important to note that while Phelps is one of the most decorated athletes in Olympic history, he did not compete in the Sydney Olympics. The most decorated athlete at the Sydney Games was Ian Thorpe, who won three gold medals and two bronze medals in swimming.

In [17]:
display(Markdown(get_response(prompt, system_prompt=system_prompt)))

The most decorated individual athlete at the Sydney 2000 Olympics was **Michael Phelps**, who won a total of **six medals**, including five golds and one bronze. However, it is essential to note that while Phelps is often recognized as one of the most decorated Olympians in history, he did not compete in the Sydney Olympics.

Instead, the athlete with the most medals in Sydney was **Ian Thorpe**, the Australian swimmer. He won a total of **five medals**: three golds in the 400m freestyle, 4x100m freestyle relay, and 4x200m freestyle relay, along with two bronze medals in the 4x100m medley relay and the 200m freestyle.

### Reasoning:
1. **Medal Count**: Thorpe's five-medal achievement was the highest for individual competitors at the Sydney Games.
  
2. **Historical Context**: Michael Phelps gained fame later in the 2004 Athens Olympics, where he began his record-setting career.

3. **Sources**: The Olympic records and official athlete statistics are used to track medal counts.

### Conclusion:
Therefore, the most decorated individual athlete at the Sydney 2000 Olympics was indeed **Ian Thorpe** with his impressive tally of five medals.